# AlgoTrading

#### Exercício 2

1. Utilizar os dados de PETR3 intradia no Blackboard
2. Usar o modelo de simulação acima usando a biblioteca backtesting
3. Adaptar a estratégia Martingale:
4. Diminuir a posição para 1 quando o preço for 3% acima do preço médio
5. Quando fizer o stop gain, estabeleça o novo preço médio como o preço de venda
6. Se o preço cair, deixe a estratégia agir, dobrando a posição
7. Interprete o resultado







In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
import random

from backtesting import evaluateHist, evaluateIntr, Strategy, Order

In [21]:
class Martingale(Strategy):

    def __init__(self):
        self.trades = []
        self.size = 0
        self.average = 0

    def receive(self, event):

        price = event.price[3] # Captura o preço atual vindo do evento

        if self.size == 0: # Se está zerado
            self.submit(self._id, Order(event.instrument, Order.B, 1, 0))
            self.size = 1
            self.average = price
            #print("{0} {1} {2}".format(price, self.average, self.size))
            
        elif (price/self.average - 1) >= 0.03: # Se está subindo 0.3% do preço médio
            self.submit(self._id, Order(event.instrument, Order.S, self.size-1, 0))
            self.size = 1
            self.average = price
            

        elif (price/self.average - 1) <= -0.005: # Se cair 0.5%, dobra
            
            # dobra a posição
            self.submit(self._id, Order(event.instrument,  Order.B, self.size, 0))
            self.size *= 2
            # novo médio
            self.average += price
            self.average /= 2
            #print("Price:{0} Average:{1} Position Quantity:{2}".format(price, self.average, self.size))
    
print(evaluateIntr(Martingale(), {'PETR4': 'PETR4-2012-04-05.csv'}))

Number of trades: 1
Gross P&L: 1.56
Gross Average Return: 1.78%
Tax: 0.04
Fee: 0.00
Net P&L: 1.52
Hitting ratio: 100.00%
Max Trade P&L: 1.56
Min Trade P&L: 1.56



#### Sobre o Resultado:

A o pipeline da estratégia é baseado em:

1. Comprar o instrumento no início do dia, independente do preço
2. Caso o valor do instrumento suba 3% em relação aquele comprado, vende
3. Caso o valor caia 0.5% é dobrado a posição


A estratégia sempre vai executar 1 trade ao dia. Isso ocorre, pois a posição nunca é zerada (ela é no mínimo 1) então não é considerado um trade ao final do dia. 

Testanto os valores para o instrumento PETR4 nos dias 3, 4 e 5 de março de 2012, observa-se uma grande variação. No dia 3, o hitting ratio é de 0%, com um lucro líquido de -20.75 reais. O mesmo ocorre para o dia 4, com um prejuízo de 6.26 reais. Já no dia 5 observa-se um ritting ratio de 100% e um lucro líquido de modestos 1.52 Reais.

Dessa maneira, a análise com dados intra-dia da estratégia mostra sua inscosistência, com uma ampla variação da sua taxa de acerto.